## 1. Import packages

In [1]:
#!pip install tweepy

  Using cached tweepy-3.10.0-py2.py3-none-any.whl (30 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)


In [3]:
pip install tqdm

     |████████████████████████████████| 74 kB 1.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:

import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## 2. Twitter API authentication

In [2]:
consumer_api_key = '5wDgSH4wltNo2lSEuAamwVbyu'
consumer_api_secret = 'oRprGgrvKOmqadCebhTo72N8EQu4moCdDgnqEx8GxgXyEOpmW0'

In [3]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [4]:
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## 3. Tweets query

### 3.1. Define the query

In [20]:
search_words = "stock -filter:retweets"
date_since = "2021-05-20"
date_until = "2021-05-29"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",geocode="40.731562,-73.871002,15km",
              since=date_since, until=date_until).items(12452)
              

### 3.2. Retreive the tweets

In [10]:
tweets_copy = []
for tweet in tqdm(tweets):
    if "NJ" not in tweet.user.location:
         tweets_copy.append(tweet)

607it [00:17, 34.45it/s]


In [11]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 607


## 4. Populate the dataset

In [12]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,
                                               #'user_description': tweet.user.description,
                                               #'user_created': tweet.user.created_at,
                                               #'user_followers': tweet.user.followers_count,
                                               #'user_friends': tweet.user.friends_count,
                                               #'user_favourites': tweet.user.favourites_count,
                                               #'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None]
                                               #'source': tweet.source,
                                               #'is_retweet': tweet.retweeted
                                              }, index=[0]))

100%|██████████| 607/607 [02:29<00:00,  4.06it/s]


In [13]:
tweets_df.head()

,user_name,user_location,date,text,hashtags
0,Fortune Tech,"New York, NY",2021-06-01 23:59:12,SoFi jumps 12% in Nasdaq debut https://t.co/Zq4qTgN2KD https://t.co/aHgWvJxd2P,None
0,The Marketing Audit,"New York, USA",2021-06-01 23:59:03,Gap vs. American Eagle Outfitters: Which Apparel Stock is a Better Buy? https://t.co/7Z8o1mPnby https://t.co/irPwC6vzu7,None
0,Kadeem Lundy,New York,2021-06-01 23:58:59,Anybody else own shares of AMC Entertainment stock?,None
0,Flash Alert,"New York, USA",2021-06-01 23:58:52,"$QDEL [15s. delayed] filed SEC form 4: SVP, R&amp;D Kroll Werner: \nDelivered securities 2,248 of Common Stock at price $121.5 and Converted securities 4,376 of Common Stock at price $0 on 2021-05-27, increased holding by 26% to 10,247 shares https://t.co/4Qb7TRD13h",None
0,Baruch Professionals Forum,New York,2021-06-01 23:58:04,Gap vs. American Eagle Outfitters: Which Apparel Stock is a Better Buy? https://t.co/hnAkc2LBT4 https://t.co/mxCTXNGJKY,None


## 5. Save the data

### 5.1. Read past data

In [17]:
tweets_old_df = pd.read_csv("/Users/okdamava/Downloads/covid-19-tweets-master/test_OKD1.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (607, 5)


### 5.2. Merge past and present data

In [18]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 607 past tweets: 607 all tweets: 1214


### 5.3. Drop duplicates

In [19]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (1214, 5)


### 5.4. Export the updated data

In [15]:
tweets_df.to_csv("test_OKD1.csv", index=False)